# CBIR System Experimentation

This notebook allows you to interactively test and visualize the Content-Based Image Retrieval (CBIR) pipeline using DWT Entropy, POPMV, and Color Statistics features.

In [ ]:
import os
import numpy as np
import cv2
from src.preprocessing import ImagePreprocessor
from src.feature_extractors import POPMVExtractor, DWTEntropyExtractor, ColorStatsExtractor
from src.fusion import FeatureFusion
from src.retriever import ImageRetriever
from utils.image_io import load_image, display_image
import matplotlib.pyplot as plt


In [ ]:
# Set up paths
config_path = '../config.json'
index_path = '../data/features/features.json'  # Update if needed
image_folder = '../data/images'
query_folder = '../data/query'

# Example parameters
popmv_params = {'window_size': 3}
dwt_params = {'wavelet': 'haar', 'level': 1}
color_params = {'color_space': 'RGB'}
fusion_params = {'normalize': True, 'norm_type': 'l2'}

In [ ]:
# Pick a sample image
sample_image_path = os.path.join(image_folder, os.listdir(image_folder)[0])

# Preprocess
preprocessor = ImagePreprocessor(config_path)
pre_img = preprocessor.preprocess(sample_image_path)

# Feature extraction
popmv = POPMVExtractor(**popmv_params)
dwt = DWTEntropyExtractor(**dwt_params)
color = ColorStatsExtractor(**color_params)

popmv_feat = popmv.extract(pre_img)
dwt_feat = dwt.extract(pre_img)
color_img = load_image(sample_image_path, color=True)
color_feat = color.extract(color_img)

print('POPMV feature:', popmv_feat)
print('DWT Entropy feature:', dwt_feat)
print('Color Stats feature:', color_feat)

In [ ]:
fusion = FeatureFusion(**fusion_params)
fused_vector = fusion.fuse([popmv_feat, dwt_feat, color_feat])
print('Fused feature vector:', fused_vector)

In [ ]:
# Test retrieval
retriever = ImageRetriever(
    config_path=config_path,
    index_path=index_path,
    popmv_params=popmv_params,
    dwt_params=dwt_params,
    color_params=color_params,
    fusion_params=fusion_params,
    distance_metric='euclidean'
)

query_image_path = os.path.join(query_folder, os.listdir(query_folder)[0])
results = retriever.retrieve(query_image_path, top_k=5)
print('Top 5 results:', results)

# Visualize query and top results
fig, axes = plt.subplots(1, 6, figsize=(18, 4))
query_img = load_image(query_image_path, color=True)
axes[0].imshow(cv2.cvtColor(query_img, cv2.COLOR_BGR2RGB))
axes[0].set_title('Query')
axes[0].axis('off')
for i, (img_id, score) in enumerate(results):
    img_path = os.path.join(image_folder, img_id)
    img = load_image(img_path, color=True)
    axes[i+1].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axes[i+1].set_title(f'Result {i+1}\nScore: {score:.3f}')
    axes[i+1].axis('off')
plt.tight_layout()
plt.show()